In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from typing import Dict, List, Tuple

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ 라이브러리 임포트 완료!")

✅ 라이브러리 임포트 완료!


In [ ]:
# ============================================================================
# 셀 2: Football 데이터 EDA - 통합 분석 (각 테이블별로 모든 분석 수행)
# ============================================================================

def _project_root() -> Path:
    cwd = Path.cwd()
    if (cwd / "data" / "raw").exists():
        return cwd
    return Path("/Users/jina/Documents/GitHub/SKN18-2nd-4Team").resolve()

def _memory_mb(df: pd.DataFrame) -> float:
    return float(df.memory_usage(deep=True).sum() / 1024**2)

def _analyze_data_leakage_risks(df: pd.DataFrame, table_name: str) -> List[str]:
    """데이터 누수 위험 요소들을 분석합니다."""
    risks = []
    
    # 1. 미래 정보 포함 가능성 (날짜 컬럼 분석)
    date_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in ['date', 'time', 'at', 'created', 'updated'])]
    if date_cols:
        risks.append(f"날짜/시간 컬럼 존재: {date_cols} - 미래 정보 누수 위험")
    
    # 2. 고유값 비율이 너무 높은 컬럼 (식별자 가능성)
    for col in df.columns:
        if df[col].dtype == 'object':
            unique_ratio = df[col].nunique() / len(df)
            if unique_ratio > 0.95:
                risks.append(f"고유값 비율 높음 ({unique_ratio:.2%}): {col} - 식별자 가능성")
    
    # 3. 결측값 패턴 분석
    missing_pattern = df.isnull().sum()
    if missing_pattern.sum() > 0:
        high_missing_cols = missing_pattern[missing_pattern > len(df) * 0.5].index.tolist()
        if high_missing_cols:
            risks.append(f"높은 결측률 컬럼: {high_missing_cols} - 데이터 품질 이슈")
    
    return risks

# 설정
PROJECT_ROOT = _project_root()
DATA_RAW_DIR = PROJECT_ROOT / "data" / "raw"
NROWS = None  # 대용량이면 예: 1_000_000 으로 설정
MAX_UNIQUE = 25  # 저카디널리티 기준

# CSV 파일 자동 탐색
csv_map = {p.stem: p for p in sorted(DATA_RAW_DIR.glob("*.csv"))}

print("=" * 80)
print("📊 FOOTBALL 데이터 EDA - 통합 분석 결과")
print("=" * 80)

# 1. 발견된 파일 목록
print("\n📁 발견된 CSV 파일 목록:")
print("-" * 50)
if not csv_map:
    print("❌ data/raw 에 CSV 파일이 없습니다.")
else:
    for i, (name, path) in enumerate(csv_map.items(), 1):
        print(f"{i:2d}. {name:<20} -> {path}")

# 2. 각 테이블별 통합 분석 (상세분석 + 저카디널리티 + 데이터 누수 위험)
print("\n" + "=" * 80)
print("📋 테이블별 통합 분석")
print("=" * 80)

all_tables_info = {}
data_leakage_risks = {}

for name, path in csv_map.items():
    try:
        print(f"\n{'='*80}")
        print(f"🔍 [{name.upper()}] 테이블 통합 분석")
        print(f"{'='*80}")
        
        df = pd.read_csv(path, nrows=NROWS, low_memory=True)
        
        # ========================================
        # 2. 테이블 상세 분석
        # ========================================
        print(f"\n📊 2. 테이블 상세 분석")
        print("-" * 50)
        
        # 기본 정보
        info = {
            'rows': len(df),
            'cols': df.shape[1],
            'memory_mb': _memory_mb(df),
            'missing_total': int(df.isnull().sum().sum()),
            'missing_rate': float(df.isnull().sum().sum() / (len(df) * len(df.columns))),
            'numeric_cols': len(df.select_dtypes(include=[np.number]).columns),
            'categorical_cols': len(df.select_dtypes(include=['object', 'category']).columns),
            'date_cols': [col for col in df.columns if any(keyword in col.lower() for keyword in ['date', 'time', 'at'])],
            'id_cols': [col for col in df.columns if any(keyword in col.lower() for keyword in ['id', 'key', 'uuid'])],
        }
        all_tables_info[name] = info
        
        print(f"📊 크기: {info['rows']:,}행, {info['cols']}열 ({info['memory_mb']:.1f}MB)")
        print(f"📈 데이터 타입: 수치형 {info['numeric_cols']}개, 범주형 {info['categorical_cols']}개")
        print(f"❌ 결측값: {info['missing_total']:,}개 ({info['missing_rate']:.2%})")
        
        if info['date_cols']:
            print(f"📅 날짜 컬럼: {info['date_cols']}")
        if info['id_cols']:
            print(f"🔑 ID 컬럼: {info['id_cols']}")
        
        # 컬럼 정보 
        print(f"\n📋 컬럼 목록 :")
        for i, col in enumerate(df.columns[:], 1):
            dtype = str(df[col].dtype)
            nunique = df[col].nunique()
            print(f"  {i:2d}. {col:<25} ({dtype:<10}) - 유니크: {nunique:,}")
        
        # 샘플 데이터
        print(f"\n📄 샘플 데이터 (상위 3행):")
        print(df.head(3).to_string())
        
        # 결측값 상세
        missing_info = df.isnull().sum()
        missing_info = missing_info[missing_info > 0].sort_values(ascending=False)
        if not missing_info.empty:
            print(f"\n⚠️  결측값 상세:")
            for col, count in missing_info.head(5).items():
                rate = count / len(df) * 100
                print(f"  {col:<25}: {count:,}개 ({rate:.1f}%)")
        else:
            print(f"\n✅ 결측값 없음")
        
        # ========================================
        # 3. 저카디널리티 컬럼 분석
        # ========================================
        print(f"\n🏷️ 3. Low 카디널리티 컬럼 분석 (유니크 ≤ {MAX_UNIQUE})")
        print("-" * 50)
        
        findings = []
        for col in df.columns:
            nunique = int(df[col].nunique(dropna=True))
            if nunique <= MAX_UNIQUE:
                vals = list(pd.Series(df[col].dropna().unique()).astype("string"))
                try:
                    vals = sorted(vals, key=lambda x: (x is None, str(x)))
                except Exception:
                    pass
                preview = ", ".join([str(v) for v in vals[:MAX_UNIQUE]])
                findings.append((col, str(df[col].dtype), nunique, preview))
        
        if findings:
            print(f"📌 저카디널리티 컬럼 발견:")
            for col, dtype_str, nunique, preview in findings:
                print(f"  {col:<25} ({dtype_str:<10}) nunique={nunique:2d} -> {preview}")
        else:
            print(f"✅ 유니크 ≤ {MAX_UNIQUE}인 컬럼이 없습니다.")
        
        # ========================================
        # 4. 데이터 누수 위험 분석
        # ========================================
        print(f"\n⚠️  4. 데이터 누수 위험 분석")
        print("-" * 50)
        
        risks = _analyze_data_leakage_risks(df, name)
        data_leakage_risks[name] = risks
        
        if risks:
            print(f"🚨 위험 요소 발견:")
            for i, risk in enumerate(risks, 1):
                print(f"  {i}. {risk}")
        else:
            print(f"✅ 위험 요소 없음")
            
    except Exception as e:
        print(f"\n❌ [{name}] 로드 실패: {e}")

📊 FOOTBALL 데이터 EDA - 통합 분석 결과

📁 발견된 CSV 파일 목록:
--------------------------------------------------
 1. appearances          -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/appearances.csv
 2. club_games           -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/club_games.csv
 3. clubs                -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/clubs.csv
 4. competitions         -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/competitions.csv
 5. game_events          -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/game_events.csv
 6. game_lineups         -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/game_lineups.csv
 7. games                -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/games.csv
 8. player_valuations    -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/player_valuations.csv
 9. players              -> /Users/jina/Documents/GitHub/SKN18-2nd-4Team/data/raw/players.csv
10. transfers            -> 

In [3]:
# 5. 전체 요약 통계
print("\n" + "=" * 80)
print("📊 전체 데이터셋 요약")
print("=" * 80)

if all_tables_info:
    total_rows = sum(info['rows'] for info in all_tables_info.values())
    total_cols = sum(info['cols'] for info in all_tables_info.values())
    total_memory = sum(info['memory_mb'] for info in all_tables_info.values())
    total_missing = sum(info['missing_total'] for info in all_tables_info.values())
    
    print(f"📈 전체 통계:")
    print(f"  - 총 테이블 수: {len(all_tables_info)}개")
    print(f"  - 총 행 수: {total_rows:,}행")
    print(f"  - 총 컬럼 수: {total_cols}개")
    print(f"  - 총 메모리 사용량: {total_memory:.1f}MB")
    print(f"  - 총 결측값: {total_missing:,}개")
    
    print(f"\n📋 테이블별 크기 순위:")
    sorted_tables = sorted(all_tables_info.items(), key=lambda x: x[1]['rows'], reverse=True)
    for i, (name, info) in enumerate(sorted_tables, 1):
        print(f"  {i:2d}. {name:<20}: {info['rows']:,}행, {info['cols']}열 ({info['memory_mb']:.1f}MB)")

print("\n" + "=" * 80)
print("✅ Football 데이터 EDA 완료!")
print("=" * 80)


📊 전체 데이터셋 요약
📈 전체 통계:
  - 총 테이블 수: 10개
  - 총 행 수: 5,765,174행
  - 총 컬럼 수: 133개
  - 총 메모리 사용량: 1916.4MB
  - 총 결측값: 1,794,715개

📋 테이블별 크기 순위:
   1. game_lineups        : 2,191,911행, 10열 (863.8MB)
   2. appearances         : 1,706,806행, 13열 (506.6MB)
   3. game_events         : 1,035,043행, 10열 (313.1MB)
   4. player_valuations   : 496,606행, 5열 (63.9MB)
   5. club_games          : 148,052행, 11열 (35.1MB)
   6. transfers           : 79,646행, 10열 (25.9MB)
   7. games               : 74,026행, 23열 (71.3MB)
   8. players             : 32,601행, 23열 (36.3MB)
   9. clubs               : 439행, 17열 (0.3MB)
  10. competitions        : 44행, 11열 (0.0MB)

✅ Football 데이터 EDA 완료!


In [4]:
# ============================================================================
# 셀 3: 추가 분석을 위한 데이터 로딩 (선택사항)
# ============================================================================

# 특정 테이블을 더 자세히 분석하고 싶다면 아래 코드를 사용하세요
# 예: appearances 테이블을 df_appearances로 로드
# df_appearances = pd.read_csv(DATA_RAW_DIR / "appearances.csv", nrows=NROWS)

print("\n💡 추가 분석 팁:")
print("- 특정 테이블을 자세히 분석하려면 해당 CSV를 개별적으로 로드하세요")
print("- 날짜 컬럼이 있다면 pd.to_datetime()으로 변환 후 시계열 분석을 고려하세요")
print("- ID 컬럼들은 조인 키로 사용될 가능성이 높으니 관계를 파악하세요")
print("- 결측값이 많은 컬럼은 데이터 품질을 개선하거나 제외를 고려하세요")


💡 추가 분석 팁:
- 특정 테이블을 자세히 분석하려면 해당 CSV를 개별적으로 로드하세요
- 날짜 컬럼이 있다면 pd.to_datetime()으로 변환 후 시계열 분석을 고려하세요
- ID 컬럼들은 조인 키로 사용될 가능성이 높으니 관계를 파악하세요
- 결측값이 많은 컬럼은 데이터 품질을 개선하거나 제외를 고려하세요
